Read in parameter values from  [``/parameters/calibration_zeta_0.75.csv``](/parameters/calibration_zeta_0.75.csv) which reads in the values for 0.75 zeta

Read in parameter values from  [``/parameters/calibration_zeta_1.25.csv``](/parameters/calibration_zeta_1.25.csv) which reads in the values for 1.25 zeta

Read in baseline parameter values from  [``/parameters/calibration_params.csv``](/parameters/calibration_params.csv)

## Setup

In [1]:
using PerlaTonettiWaugh, Parameters, Suppressor

In [2]:
params = load_parameters("parameters/calibration_params.csv")
d = params.d_0
d_lowered = 1 + 0.90*(d-1)

ζ_075 = load_parameters("parameters/calibration_zeta_0.75.csv")
ζ_1 = load_parameters("parameters/calibration_params.csv") # baseline 
ζ_125 = load_parameters("parameters/calibration_zeta_1.25.csv") 

ζ_075 = merge(ζ_075, (d = d,))
ζ_1 = merge(ζ_1, (d = d,))
ζ_125 = merge(ζ_125, (d = d,)) 

ζ_075_lowered = merge(ζ_075, (d = d_lowered,))
ζ_1_lowered = merge(ζ_1, (d = d_lowered,)) # baseline
ζ_125_lowered = merge(ζ_125, (d = d_lowered,));

## Zeta 1

### Baseline Allocations

In [3]:
sol_1 = stationary_algebraic(ζ_1, settings_defaults())
for key in (:g, :Ω, :z_hat, :λ_ii, :c)
    val = sol_1[key]
    println("$(key) = $(val)")
end

g = 0.007913401963163308
Ω = 0.6805904609927759
z_hat = 1.9813268967445312
λ_ii = 0.893708728294921
c = 0.8706834939103555


### Counterfactual (Lowered) Allocations and Consumption Equivalent

In [4]:
sol_1_lowered = stationary_algebraic(ζ_1_lowered, settings_defaults())
for key in (:g, :Ω, :z_hat, :λ_ii, :c)
    val = sol_1_lowered[key]
    println("$(key) = $(val)")
end

g = 0.010250822794504864
Ω = 0.6459867736680422
z_hat = 1.8456817342356477
λ_ii = 0.8555713586729671
c = 0.8629419672035096


In [5]:
ce_1 = 100*consumption_equivalent(sol_1_lowered.U_bar, sol_1.U_bar, ζ_1)

11.181899506569803

### Transition Dynamics and Consumption Equivalent

In [6]:
overall_parameters = merge(ζ_1, (d_0 = d, d_T = d_lowered,));
settings = settings_defaults()
@unpack stationary_T, stationary_0, change_welfare, change_trade = compare_steady_states(overall_parameters, settings);

In [7]:
abs(stationary_T.g - sol_1_lowered.g) < 1e-5 && abs(stationary_0.g - sol_1.g) < 1e-5 # check that the grid is close enough

true

In [8]:
cachename = model_cachename(overall_parameters, settings) # uniquely identify the run by parameters + settings
cachepath = joinpath(pwd(), "data", cachename * ".csv")
use_cache = true # change this to false if you don't want to use the cache for a run

if isfile(cachepath) && use_cache # read if possible
    println("using cache.")
    solved = CSV.read(cachepath);
else
    println("solving without cache.")
    @time solution = @suppress solve_transition(overall_parameters, settings);
    solved = solution.results;
end

solving without cache.
599.276696 seconds (2.11 G allocations: 720.224 GiB, 28.25% gc time)


,t,g,z_hat,Ω,E,v_1,L_tilde,entry_residual
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-20.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,2.08367e-12
2,-15.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,2.08367e-12
3,-10.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,2.08367e-12
4,-5.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,2.08367e-12
5,-1.0,0.00792017,1.9882,0.685242,0.02,6.86678,0.242265,2.08367e-12
6,0.0,0.00903988,1.87401,0.685242,0.0156134,6.86678,0.229549,0.0
7,0.25,0.00904654,1.87389,0.684969,0.0156577,6.86678,0.229729,0.0
8,0.5,0.0090532,1.87377,0.684697,0.0157018,6.86678,0.229908,0.0
9,0.75,0.00905986,1.87364,0.684425,0.0157457,6.86678,0.230085,0.0


In [9]:
filter!(row -> row.t >= 0, solved) # don't plot pre-shock values
@show ce_1_transition = 100*consumption_equivalent(solved.U[1], stationary_0.U_bar, overall_parameters);

ce_1_transition = 100 * consumption_equivalent(solved.U[1], stationary_0.U_bar, overall_parameters) = 10.88395069226058


## Zeta 0.75

### Baseline Allocations

In [10]:
sol_075 = stationary_algebraic(ζ_075, settings_defaults())
for key in (:g, :Ω, :z_hat, :λ_ii, :c)
    val = sol_075[key]
    println("$(key) = $(val)")
end

g = 0.007913401963163575
Ω = 0.9074539479905955
z_hat = 1.9813268967445012
λ_ii = 0.8937087282949169
c = 0.9943001206514349


### Counterfactual (Lowered) Allocations and Consumption Equivalent

In [11]:
sol_075_lowered = stationary_algebraic(ζ_075_lowered, settings_defaults())
for key in (:g, :Ω, :z_hat, :λ_ii, :c)
    val = sol_075_lowered[key]
    println("$(key) = $(val)")
end

g = 0.010250822798545837
Ω = 0.8613156829790793
z_hat = 1.8456817342226686
λ_ii = 0.8555713586705149
c = 0.9854594763818336


In [12]:
ce_075 = 100*consumption_equivalent(sol_075_lowered.U_bar, sol_075.U_bar, ζ_075)

11.181899256227856

### Transition Dynamics and Consumption Equivalent

In [13]:
overall_parameters = merge(ζ_075, (d_0 = d, d_T = d_lowered,));
settings = settings_defaults()
@unpack stationary_T, stationary_0, change_welfare, change_trade = compare_steady_states(overall_parameters, settings);

In [14]:
abs(stationary_T.g - sol_075_lowered.g) < 1e-5 && abs(stationary_0.g - sol_075.g) < 1e-5 # check that the grid is close enough

true

In [15]:
cachename = model_cachename(overall_parameters, settings) # uniquely identify the run by parameters + settings
cachepath = joinpath(pwd(), "data", cachename * ".csv")
use_cache = true # change this to false if you don't want to use the cache for a run

if isfile(cachepath) && use_cache # read if possible
    println("using cache.")
    solved = CSV.read(cachepath);
else
    println("solving without cache.")
    @time solution = @suppress solve_transition(overall_parameters, settings);
    solved = solution.results;
end

filter!(row -> row.t >= 0, solved) # don't plot pre-shock values
@show ce_075_transition = 100*consumption_equivalent(solved.U[1], stationary_0.U_bar, overall_parameters);

solving without cache.
492.877372 seconds (1.49 G allocations: 670.730 GiB, 33.21% gc time)
ce_075_transition = 100 * consumption_equivalent(solved.U[1], stationary_0.U_bar, overall_parameters) = 10.80213236441383


## Zeta 1.25

### Baseline Allocations

In [16]:
sol_125 = stationary_algebraic(ζ_125, settings_defaults())
for key in (:g, :Ω, :z_hat, :λ_ii, :c)
    val = sol_125[key]
    println("$(key) = $(val)")
end

g = 0.007913401963146154
Ω = 0.5444723687853136
z_hat = 1.98132689674432
λ_ii = 0.8937087282948925
c = 0.7854850570129526


### Counterfactual (Lowered) Allocations and Consumption Equivalent

In [17]:
sol_125_lowered = stationary_algebraic(ζ_125_lowered, settings_defaults())
for key in (:g, :Ω, :z_hat, :λ_ii, :c)
    val = sol_125_lowered[key]
    println("$(key) = $(val)")
end

g = 0.01025082279854586
Ω = 0.5167894201216133
z_hat = 1.8456817342226666
λ_ii = 0.8555713586705145
c = 0.7785010572181699


In [18]:
ce_125 = 100*consumption_equivalent(sol_125_lowered.U_bar, sol_125.U_bar, ζ_125)

11.181899562200105

### Transition Dynamics and Consumption Equivalent

In [19]:
overall_parameters = merge(ζ_125, (d_0 = d, d_T = d_lowered,));
settings = settings_defaults()
@unpack stationary_T, stationary_0, change_welfare, change_trade = compare_steady_states(overall_parameters, settings);

In [20]:
abs(stationary_T.g - sol_125_lowered.g) < 1e-5 && abs(stationary_0.g - sol_125.g) < 1e-5 # check that the grid is close enough

true

In [21]:
## commenting out as this doesn't solve well
# cachename = model_cachename(overall_parameters, settings) # uniquely identify the run by parameters + settings
# cachepath = joinpath(pwd(), "data", cachename * ".csv")
# use_cache = true # change this to false if you don't want to use the cache for a run

# if isfile(cachepath) && use_cache # read if possible
#     println("using cache.")
#     solved = CSV.read(cachepath);
# else
#     println("solving without cache.")
#     @time solution = @suppress solve_transition(overall_parameters, settings);
#     solved = solution.results;
# end

# filter!(row -> row.t >= 0, solved) # don't plot pre-shock values
# @show ce_125_transition = 100*consumption_equivalent(solved.U[1], stationary_0.U_bar, overall_parameters);